In [9]:
url = 'https://www.bilibili.com/video/BV1Gj41127qW/?spm_id_from=333.1007.tianma.1-2-2.click&vd_source=3d29284fd9e36332dce3769745b61ac2'

print(url.split('/')[-2])

BV1Gj41127qW


In [32]:
# 今日头条搜索框搜索后视频爬取下载
# 线程池并行视频保存（时间19.6s）

from multiprocessing.dummy import Pool
from time import sleep
import random
import requests
import urllib.request
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

import bilibili



class jinritoutiao():

    def __init__(self):
        self.url = ""

    def fun_url(self, web, keys):
        # 随机返回一个user_agent进行UI伪装
        global bilibili

        def get_one_user_agent():
            with open('./user_agent_all_ok.txt', 'r', encoding='utf-8') as file:
                ret = file.readlines()
            user_agent_list = [i.strip() for i in ret]
            one_user_agent = random.choice(user_agent_list)
            return one_user_agent

        # headers = {
        #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
        # }


        web_url = ''
        search_btn_url = ''
        video_list_url = ''
        if web == '今日头条':
            web_url = 'https://www.toutiao.com/'
            search_btn_url = '//*[@id="root"]/div/div[4]/div/div[1]/button'
            video_list_url = '//*[@class="cs-view cs-view-block cs-grid-cell grid-cell-3 grid-cell-x-m grid-cell-y-m"]/div/div/div[1]'

        elif web == '哔哩哔哩':
            web_url = 'https://www.bilibili.com/'
            search_btn_url = '//*[@id="nav-searchform"]/div[2]'
            video_list_url = '//*[@class="video-item matrix"]/a'

        print(web_url)
        print(search_btn_url)

        one_user_agent = get_one_user_agent()
        # print(one_user_agent)
        headers = {'User-Agent': one_user_agent}

        option = webdriver.ChromeOptions()
        # #设置无头模式
        # option.add_argument("--headless")

        # 关闭左上方 Chrome 正受到自动测试软件的控制的提示
        option.add_experimental_option('excludeSwitches', ['enable-automation'])
        option.add_experimental_option('useAutomationExtension', False)

        # 设置user-agent
        # option.add_argument('user-agent=' + headers['User-Agent'])
        option.add_argument('user-agent=' + one_user_agent)

        # driver = webdriver.Chrome(options=option)
        s = Service('./chromedriver.exe')
        driver = webdriver.Chrome(service=s, options=option)
        # driver = webdriver.Chrome(service=s)

        driver.get(web_url)
        # sleep(2)

        driver.implicitly_wait(2)

        # 主页搜索框搜索.....
        # search_input = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[4]/div/div[1]/input')
        search_input = driver.find_element(By.XPATH, '//input')
        search_input.send_keys(keys)

        # 主页搜索按钮点击
        btn = driver.find_element(By.XPATH, value=search_btn_url)
        btn.click()
        # sleep(2)
        driver.implicitly_wait(2)

        # print(search_input)

        # 更新driver页面数据到新的页面（主界面——搜索界面）
        handle = driver.window_handles
        driver.switch_to.window(handle[-1])

        # 更新url到现在页面——搜索界面
        # url = driver.current_url
        # driver.get(url)
        # print(url)
        # sleep(2)
        driver.implicitly_wait(2)

        handle_main = driver.current_window_handle  # 句柄

        # page_text = driver.page_source
        # with open('./军事/源码.html', 'w', encoding='utf-8') as fp:
        #     fp.write(page_text)
        # # print(page_text)

        # 今日头条点击到“视频”界面
        if web == '今日头条':
            video_btn = driver.find_element(By.XPATH, value='//*[@class="cs-view cs-view-flex align-items-center flex-row"]/a[3]')
            video_btn.click()
            # sleep(2)
            driver.implicitly_wait(2)

        video_list = driver.find_elements(By.XPATH, value=video_list_url)
        # video_list = driver.find_elements(By.XPATH, value='//*[@class="cs-view cs-view-block cs-grid-cell grid-cell-3 grid-cell-x-m grid-cell-y-m"//@href')
        index = 0
        urls = []  # 存储所有视频的链接和文件名
        bids = []

        for video in video_list:
            video.click()
            a = random.random() * 3
            sleep(a)  # 设置随机休息

            # 更新driver页面数据到新的页面
            handle = driver.window_handles
            driver.switch_to.window(handle[-1])

            if web == '哔哩哔哩':
                # 更新url到现在页面
                video_page_url = driver.current_url
                print(video_page_url)
                bid = video_page_url.split('/')[-2]
                print(bid)
                bids.append(bid)
                # driver.get(video_page_url)
                index += 1


            elif web == '今日头条':

                page_text = driver.page_source
                tree = etree.HTML(page_text)
                src_url = 'https:' + tree.xpath('//video/@src')[0]
                print(src_url)

                title = tree.xpath('//h1/@title | //h2/@title | //h3/@title')[0] + '.mp4'

                if src_url.find('blob') == -1:
                    # response = requests.get(url=src_url, headers=headers)
                    # urllib.request.urlretrieve(src_url, './视频/' + title)
                    # print(title + ".mp4视频文件下载完成！")

                    dic = {
                        'title': title,
                        'url': src_url
                    }
                    urls.append(dic)
                    index += 1

            if index == 3:
                print("视频链接保存完成！")
                driver.close()
                # driver.quit()
                break

            driver.close()
            driver.switch_to.window(handle_main)

        print(bids)
        if web == '哔哩哔哩':
            for bid in bids:
                bilibili = bilibili.BilibiliVideoAudio(bid)
                video_audio_info = bilibili.get_video_audio()

                title = video_audio_info['title']
                video_url = video_audio_info['video_url']
                audio_url = video_audio_info['audio_url']

                # print(video_url)
                # print(audio_url)

                # bilibili.download_video_audio(video_url, f"{title}.mp4")  # 下载视频
                # bilibili.download_video_audio(audio_url, f"{title}.mp3")  # 下载音频

                dic = {
                    'title': title,
                    'video_url': video_url
                    'audio_url': audio_url
                }
                urls.append(dic)

        return urls
    
c = jinritoutiao()
urls = c.fun_url(web='哔哩哔哩', keys='乌克兰')
# urls = c.fun_url(web='哔哩哔哩', keys='乌克兰')





https://www.bilibili.com/
//*[@id="nav-searchform"]/div[2]
https://www.bilibili.com/video/BV1bH4y1Q7RL/?from=search&seid=11463572555927226702&spm_id_from=333.337.0.0
BV1bH4y1Q7RL
https://www.bilibili.com/video/BV1Cu4y1C7Xm/?from=search&seid=11463572555927226702&spm_id_from=333.337.0.0
BV1Cu4y1C7Xm
https://www.bilibili.com/video/BV12h4y1m7CD/?from=search&seid=11463572555927226702&spm_id_from=333.337.0.0
BV12h4y1m7CD
视频链接保存完成！
['BV1bH4y1Q7RL', 'BV1Cu4y1C7Xm', 'BV12h4y1m7CD']


UnboundLocalError: local variable 'src_url' referenced before assignment

In [18]:
def get_video_data(dic):
    one_user_agent = get_one_user_agent()
    # print(one_user_agent)
    headers = {'User-Agent': one_user_agent}
    url = dic['url']
    print(dic['title'], '正在下载......')
    data = requests.get(url=url, headers=headers).content
    with open(dic['title'], 'wb') as fp:
        fp.write(data)
        print(dic['title'], '下载成功')

pool = Pool(4)
pool.map(get_video_data, urls)

pool.close()
pool.join()

In [26]:
# 今日头条搜索框搜索后视频爬取下载
# 线程池并行视频保存（时间19.6s）

from multiprocessing.dummy import Pool
from time import sleep
import random
import requests
import urllib.request
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# headers = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
# }

def get_one_user_agent():
    with open('./user_agent_all_ok.txt', 'r', encoding='utf-8') as file:
        ret = file.readlines()
    user_agent_list = [i.strip() for i in ret]
    one_user_agent = random.choice(user_agent_list)
    return one_user_agent	

    
one_user_agent = get_one_user_agent()
print(one_user_agent)
headers={'User-Agent': one_user_agent}

option = webdriver.ChromeOptions()
# #设置无头模式
# option.add_argument("--headless")

# 关闭左上方 Chrome 正受到自动测试软件的控制的提示
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_experimental_option('useAutomationExtension', False)

#设置user-agent
# option.add_argument('user-agent=' + headers['User-Agent'])
option.add_argument('user-agent=' + one_user_agent)


# driver = webdriver.Chrome(options=option)
s = Service('./chromedriver.exe')
driver = webdriver.Chrome(service=s,options=option)
# driver = webdriver.Chrome(service=s)


driver.get('https://www.bilibili.com/')
# sleep(2)

driver.implicitly_wait(2)

# 搜索框搜索.....
search_input = driver.find_element(By.XPATH, '//input')
search_input.send_keys('乌克兰')

# 搜索按钮点击
btn = driver.find_element(By.XPATH, value='//*[@id="nav-searchform"]/div[2]')
btn.click()
# sleep(2)
driver.implicitly_wait(2)

# print(search_input)


# 更新driver页面数据到新的页面
handle = driver.window_handles
driver.switch_to.window(handle[-1])

# 更新url到现在页面
# url = driver.current_url
# driver.get(url)
# print(url)
# sleep(2)
driver.implicitly_wait(2)

handle_main = driver.current_window_handle  # 句柄

video_list = driver.find_elements(By.XPATH, value='//*[@class="video-item matrix"]/a')
# video_list = driver.find_elements(By.XPATH, value='//*[@class="cs-view cs-view-block cs-grid-cell grid-cell-3 grid-cell-x-m grid-cell-y-m"//@href')


Mozilla5.0 (Windows NT 6.1; WOW64) AppleWebKit536.3 (KHTML, like Gecko) Chrome19.0.1062.0 Safari536.3


In [27]:
index = 0
urls = []  # 存储所有视频的链接和文件名

for video in video_list:
    video.click()
    a = random.random() * 3
    sleep(a)  # 设置随机休息

    # 更新driver页面数据到新的页面
    handle=driver.window_handles
    driver.switch_to.window(handle[-1])

    # 更新url到现在页面
    # video_page_url = driver.current_url
    # driver.get(video_page_url)

    page_text = driver.page_source
    tree = etree.HTML(page_text)
    src_url = 'https:' + tree.xpath('//video/@src')[0]
    print(src_url)

    title = tree.xpath('//h1/@title')[0] + '.mp4'

    if src_url.find('blob') == -1:
        # response = requests.get(url=src_url, headers=headers)
        # urllib.request.urlretrieve(src_url, './视频/' + title)
        # print(title + ".mp4视频文件下载完成！")
        
        dic = {
            'title' : title,
            'url' : src_url
        }
        urls.append(dic)
        index += 1

    if index == 3:

        print("视频链接保存完成！")
        driver.close()
        # driver.quit()
        break

    driver.close()
    driver.switch_to.window(handle_main)


# 对视频链接发起秦秋获取视频的二进制数据，然后将视频数据进行返回
def get_video_data(dic):
    url = dic['url']
    print(dic['title'],'正在下载......')
    data = requests.get(url= url,headers=headers).content
    with open(dic['title'],'wb') as fp:
        fp.write(data)
        print(dic['title'],'下载成功')


pool = Pool(4)
pool.map(get_video_data, urls)

pool.close()
pool.join()

https:blob:https://www.bilibili.com/6acecb9d-696c-4014-a65b-ce51f4bb40f7
https:blob:https://www.bilibili.com/00eef86b-2606-4010-9a11-38ab4bf9633a
https:blob:https://www.bilibili.com/f1ef517f-f0f2-4395-82a8-1ba415bacc8c
https:blob:https://www.bilibili.com/2d68772d-75a1-455c-9c1b-534826ec7eaf
https:blob:https://www.bilibili.com/ecd0efa3-f729-4052-8b70-26ff89000c01
https:blob:https://www.bilibili.com/f548baf2-db69-48f0-b4ee-bdb67007e59c
https:blob:https://www.bilibili.com/701d6052-856b-4527-ab45-03846206e377
https:blob:https://www.bilibili.com/25677edb-35d5-463f-89a0-c9c09ec5eebd
https:blob:https://www.bilibili.com/33770cce-263f-41fb-b2e9-cc282efb7422


KeyboardInterrupt: 